In [32]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import os
import re


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [33]:
# model setting
max_len = 50
hidden_units = 50
num_heads = 1
num_layers = 2
dropout_rate=0.5
num_workers = 1
device = 'cuda' 

# training setting
lr = 0.001
batch_size = 128
num_epochs = 200
mask_prob = 0.15 # for cloze task

In [34]:
############# 중요 #############
# data_path는 사용자의 디렉토리에 맞게 설정해야 합니다.
data_path = '/opt/ml/input/data/train'
train_rating = os.path.join(data_path, "train_ratings.csv")
genres = os.path.join(data_path, "genres.tsv")
titles = os.path.join(data_path, "titles.tsv")
writers = os.path.join(data_path, "writers.tsv")
years = os.path.join(data_path, "years.tsv")
directors = os.path.join(data_path, "directors.tsv")

df = pd.read_csv(train_rating)
df_genres = pd.read_csv(genres, delimiter='\t')
df_titles = pd.read_csv(titles, delimiter='\t')
df_writers = pd.read_csv(writers, delimiter='\t')
df_years = pd.read_csv(years, delimiter='\t')
df_directors = pd.read_csv(directors, delimiter='\t')

In [35]:
# genre dummy화

df_genres = pd.get_dummies(df_genres, columns=['genre'])
genres_value = df_genres[df_genres.duplicated(['item'], keep=False)].values.tolist()
genres_column = df_genres.columns.tolist()
genres_value.sort()
genres_values = defaultdict(list)
for i in genres_value:
    genres_values[i[0]].append(i[1:])
for i in genres_values:
    tmp = [0 for _ in range(len(genres_values[i][0]))]
    for j in genres_values[i]:
        for k in range(len(tmp)):
            tmp[k] += j[k]
    genres_values[i] = tmp
genres_value_ = []
for i in genres_values:
    genres_value_.append([i] + genres_values[i])

df_genres = df_genres[~df_genres.duplicated(['item'], keep=False)].values.tolist() + genres_value_
df_genres = pd.DataFrame(df_genres, columns = genres_column)
df_genres.head()

,item,genre_Action,genre_Adventure,genre_Animation,genre_Children,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,genre_Fantasy,genre_Film-Noir,genre_Horror,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,genre_Western
0,1193,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1258,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,457,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1961,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,2918,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
# writer dummy화

df_writers = pd.get_dummies(df_writers, columns=['writer'])
writers_value = df_writers[df_writers.duplicated(['item'], keep=False)].values.tolist()
writers_column = df_writers.columns.tolist()
writers_value.sort()
writers_values = defaultdict(list)
for i in writers_value:
    writers_values[i[0]].append(i[1:])
for i in writers_values:
    tmp = [0 for _ in range(len(writers_values[i][0]))]
    for j in writers_values[i]:
        for k in range(len(tmp)):
            tmp[k] += j[k]
    writers_values[i] = tmp
writers_value_ = []
for i in writers_values:
    writers_value_.append([i] + writers_values[i])

df_writers = df_writers[~df_writers.duplicated(['item'], keep=False)].values.tolist() + writers_value_
df_writers = pd.DataFrame(df_writers, columns = writers_column)
df_writers.head()

,item,writer_nm0000005,writer_nm0000019,writer_nm0000033,writer_nm0000036,writer_nm0000040,writer_nm0000041,writer_nm0000045,writer_nm0000059,writer_nm0000076,...,writer_nm3890871,writer_nm4160687,writer_nm4611078,writer_nm4950667,writer_nm4951717,writer_nm5022110,writer_nm5335213,writer_nm5371819,writer_nm5927607,writer_nm5927608
0,1237,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5147,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7327,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2068,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7396,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# director dummy화

df_directors = pd.get_dummies(df_directors, columns=['director'])
directors_value = df_directors[df_directors.duplicated(['item'], keep=False)].values.tolist()
directors_column = df_directors.columns.tolist()
directors_value.sort()
directors_values = defaultdict(list)
for i in directors_value:
    directors_values[i[0]].append(i[1:])
for i in directors_values:
    tmp = [0 for _ in range(len(directors_values[i][0]))]
    for j in directors_values[i]:
        for k in range(len(tmp)):
            tmp[k] += j[k]
    directors_values[i] = tmp
directors_value_ = []
for i in directors_values:
    directors_value_.append([i] + directors_values[i])

df_directors = df_directors[~df_directors.duplicated(['item'], keep=False)].values.tolist() + directors_value_
df_directors = pd.DataFrame(df_directors, columns = directors_column)
df_directors.head()

,item,director_nm0000005,director_nm0000019,director_nm0000033,director_nm0000036,director_nm0000037,director_nm0000040,director_nm0000041,director_nm0000045,director_nm0000059,...,director_nm2284484,director_nm2304017,director_nm2320658,director_nm2480587,director_nm2482088,director_nm2588606,director_nm2648685,director_nm2676052,director_nm2879822,director_nm9054338
0,1237,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5147,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7327,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2068,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7396,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# data side information / genres, titles, directors, writers, years
side_info = pd.merge(df_genres, df_titles, how ="outer", on='item')
side_info = side_info.merge(df_directors, how="outer", on='item')
side_info = side_info.merge(df_writers, how='outer', on='item')
side_info = side_info.merge(df_years, how='outer',on = 'item')

In [39]:
# year 결측치 title에서 찾음

f = re.compile("\d{4}")

side_info['year_new'] = np.where(pd.notnull(side_info['year'])==True, side_info['year'], side_info["title"].apply(lambda x: f.findall(x)[-1]))

In [41]:
side_info = side_info.drop(columns=['year', 'title'], axis = 1) # 연도, 제목 제거
side_info = side_info.fillna(0) # 결측치 0으로 대체
side_info = side_info.astype(int) # 문자형 int로 변환
side_info

,item,genre_Action,genre_Adventure,genre_Animation,genre_Children,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,genre_Fantasy,...,writer_nm4160687,writer_nm4611078,writer_nm4950667,writer_nm4951717,writer_nm5022110,writer_nm5335213,writer_nm5371819,writer_nm5927607,writer_nm5927608,year_new
0,1193,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1975
1,1258,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1980
2,457,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1993
3,1961,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1988
4,2918,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6802,117176,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2014
6803,118696,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2014
6804,118997,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2014
6805,119141,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2014


In [48]:

item_ids = df['item'].unique()
user_ids = df['user'].unique()
num_item, num_user = len(item_ids), len(user_ids)
num_batch = num_user // batch_size

# user, item indexing
item2idx = pd.Series(data=np.arange(len(item_ids))+1, index=item_ids) # item re-indexing (1~num_item), num_item+1: mask idx
user2idx = pd.Series(data=np.arange(len(user_ids)), index=user_ids) # user re-indexing (0~num_user-1)

# dataframe indexing
df = pd.merge(df, pd.DataFrame({'item': item_ids, 'item_idx': item2idx[item_ids].values}), on='item', how='inner')
df = pd.merge(df, pd.DataFrame({'user': user_ids, 'user_idx': user2idx[user_ids].values}), on='user', how='inner')
df.sort_values(['user_idx', 'time'], inplace=True)
del df['item'], df['user'] 

# train set, valid set 생성
users = defaultdict(list) # defaultdict은 dictionary의 key가 없을때 default 값을 value로 반환
user_train = {}
user_valid = {}
for u, i, t in zip(df['user_idx'], df['item_idx'], df['time']):
    users[u].append(i)

for user in users:
    user_train[user] = users[user][:-1]
    user_valid[user] = [users[user][-1]]

print(f'num users: {num_user}, num items: {num_item}')

num users: 31360, num items: 6807


In [4]:
class SeqDataset(Dataset):
    def __init__(self, user_train, num_user, num_item, max_len, mask_prob):
        self.user_train = user_train
        self.num_user = num_user
        self.num_item = num_item
        self.max_len = max_len
        self.mask_prob = mask_prob

    def __len__(self):
        # 총 user의 수 = 학습에 사용할 sequence의 수
        return self.num_user

    def __getitem__(self, user): 
        # iterator를 구동할 때 사용됩니다.
        seq = self.user_train[user]
        tokens = []
        labels = []
        for s in seq:
            prob = np.random.random() # TODO1: numpy를 사용해서 0~1 사이의 임의의 값을 샘플링하세요.
            if prob < self.mask_prob:
                prob /= self.mask_prob

                # BERT 학습
                if prob < 0.8:
                    # masking
                    tokens.append(self.num_item + 1)  # mask_index: num_item + 1, 0: pad, 1~num_item: item index
                elif prob < 0.9:
                    tokens.append(np.random.randint(1, self.num_item+1))  # item random sampling
                else:
                    tokens.append(s)
                labels.append(s)  # 학습에 사용
            else:
                tokens.append(s)
                labels.append(0)  # 학습에 사용 X, trivial
        tokens = tokens[-self.max_len:]
        labels = labels[-self.max_len:]
        mask_len = self.max_len - len(tokens)

        # zero padding
        tokens = [0] * mask_len + tokens
        labels = [0] * mask_len + labels
        return torch.LongTensor(tokens), torch.LongTensor(labels)

In [5]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(ScaledDotProductAttention, self).__init__()
        self.hidden_units = hidden_units
        self.dropout = nn.Dropout(dropout_rate) # dropout rate

    def forward(self, Q, K, V, mask):
        attn_score = torch.matmul(Q, K.transpose(2, 3)) / math.sqrt(self.hidden_units)
        attn_score = attn_score.masked_fill(mask == 0, -1e9)  # 유사도가 0인 지점은 -infinity로 보내 softmax 결과가 0이 되도록 함
        attn_dist = self.dropout(F.softmax(attn_score, dim=-1))  # attention distribution
        output = torch.matmul(attn_dist, V)  # dim of output : batchSize x num_head x seqLen x hidden_units
        return output, attn_dist

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads # head의 수
        self.hidden_units = hidden_units
        
        # query, key, value, output 생성을 위해 Linear 모델 생성
        self.W_Q = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_K = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_V = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_O = nn.Linear(hidden_units, hidden_units, bias=False)

        self.attention = ScaledDotProductAttention(hidden_units, dropout_rate) # scaled dot product attention module을 사용하여 attention 계산
        self.dropout = nn.Dropout(dropout_rate) # dropout rate
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, enc, mask):
        residual = enc # residual connection을 위해 residual 부분을 저장
        batch_size, seqlen = enc.size(0), enc.size(1)
        
        # Query, Key, Value를 (num_head)개의 Head로 나누어 각기 다른 Linear projection을 통과시킴
        Q = self.W_Q(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units) 
        K = self.W_K(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        V = self.W_V(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)

        # Head별로 각기 다른 attention이 가능하도록 Transpose 후 각각 attention에 통과시킴
        Q, K, V = Q.transpose(1, 2), K.transpose(1, 2), V.transpose(1, 2)
        output, attn_dist = self.attention(Q, K, V, mask)

        # 다시 Transpose한 후 모든 head들의 attention 결과를 합칩니다.
        output = output.transpose(1, 2).contiguous() 
        output = output.view(batch_size, seqlen, -1)

        # Linear Projection, Dropout, Residual sum, and Layer Normalization
        output = self.layerNorm(self.dropout(self.W_O(output)) + residual)
        return output, attn_dist
    
class PositionwiseFeedForward(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(PositionwiseFeedForward, self).__init__()
        
        # SASRec과의 dimension 차이가 있습니다.
        self.W_1 = nn.Linear(hidden_units, 4 * hidden_units) 
        self.W_2 = nn.Linear(4 * hidden_units, hidden_units)
        self.dropout = nn.Dropout(dropout_rate)
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, x):
        residual = x
        output = self.W_2(F.gelu(self.dropout(self.W_1(x)))) # activation: relu -> gelu
        output = self.layerNorm(self.dropout(output) + residual)
        return output
    
class BERT4RecBlock(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(BERT4RecBlock, self).__init__()
        self.attention = MultiHeadAttention(num_heads, hidden_units, dropout_rate)
        self.pointwise_feedforward = PositionwiseFeedForward(hidden_units, dropout_rate)

    def forward(self, input_enc, mask):
        output_enc, attn_dist = self.attention(input_enc, mask)
        output_enc = self.pointwise_feedforward(output_enc)
        return output_enc, attn_dist

In [ ]:
class NOVA(nn.Module):
    def __init__(self, num_user, num_item, hidden_units, num_heads, num_layers, max_len, dropout_rate, device):
        super(NOVA, self).__init__()

        self.num_user = num_user
        self.num_item = num_item
        self.hidden_units = hidden_units
        self.num_heads = num_heads
        self.num_layers = num_layers 
        self.device = device
        
        self.item_emb = nn.Embedding(num_item + 2, hidden_units, padding_idx=0) # TODO2: mask와 padding을 고려하여 embedding을 생성해보세요.
        self.pos_emb = nn.Embedding(max_len, hidden_units) # learnable positional encoding
        self.dropout = nn.Dropout(dropout_rate)
        self.emb_layernorm = nn.LayerNorm(hidden_units, eps=1e-6)
        
        self.blocks = nn.ModuleList([BERT4RecBlock(num_heads, hidden_units, dropout_rate) for _ in range(num_layers)])
        self.out = nn.Linear(hidden_units, num_item + 1) # TODO3: 예측을 위한 output layer를 구현해보세요. (num_item 주의)
        
    def forward(self, log_seqs):
        seqs = self.item_emb(torch.LongTensor(log_seqs).to(self.device))
        positions = np.tile(np.array(range(log_seqs.shape[1])), [log_seqs.shape[0], 1])
        seqs += self.pos_emb(torch.LongTensor(positions).to(self.device))
        seqs = self.emb_layernorm(self.dropout(seqs))

        mask = torch.BoolTensor(log_seqs > 0).unsqueeze(1).repeat(1, log_seqs.shape[1], 1).unsqueeze(1).to(self.device) # mask for zero pad
        for block in self.blocks:
            seqs, attn_dist = block(seqs, mask)
        out = self.out(seqs)
        return out